In [44]:
!pip install tweepy
import tweepy
import json
import time
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAABoebgEAAAAA8nOwpOH6GviG0pVFbGAEowC6lrE%3DvQmBGiPwQ2EJKcoMQUNYDwgb0d5vkjxvSPtkuqY0FFGY9D2yS8", wait_on_rate_limit=True)


    
import os
import pandas as pd
import json
import glob
from itertools import chain
import math
import numpy as np
file = "test.data.txt"
file_open = open('./project-data/' + file, 'r')
count = 0
delta = pd.DataFrame()
for line in file_open.readlines():
    tweet_data={}
    line = line.strip()
    each_tweet = line.split(',')
    main_tweet_id = each_tweet[0]
    main_tweet = client.get_tweet(main_tweet_id,expansions="author_id",user_fields=["verified","public_metrics"])
    each_tweet = each_tweet[1:]
    # count+=len(each_tweet)
    if len(each_tweet) > 100:
        num_sub_list = math.ceil(len(each_tweet)/100)
        a = np.array_split(each_tweet, num_sub_list)
        tweets = []
        for sub_list in a:
            sub_list = sub_list.tolist()
            tw = client.get_tweets(sub_list, tweet_fields=["created_at"], expansions="author_id",user_fields=["verified","public_metrics"])
            tweets.append(tw.data)
        
        tweets =  list(chain.from_iterable(tweets))

        # tweets = client.get_tweets()
    elif len(each_tweet) <= 100 and len(each_tweet) > 0:
        tweets = client.get_tweets(each_tweet,tweet_fields=["created_at"],expansions="author_id", user_fields=["verified","public_metrics"])
 
        tweets = tweets.data if tweets.data is not None else []
    replies = []
    if len(tweets) > 0:
        for tweet in tweets:
            reply_data = {}
            reply_data['tweet_id'] = tweet.id
            reply_data['tweet'] = tweet.text
            reply_data['created_at'] = tweet.created_at.timestamp()

            replies.append(reply_data)

    if main_tweet.data is not None:
        tweet_data['main_tweet'] = main_tweet.data.text
        tweet_data['main_tweet_id'] = main_tweet.data.id
        tweet_data['verified'] = main_tweet.includes["users"][0].verified
        tweet_data['followers'] = main_tweet.includes["users"][0].public_metrics['followers_count']

    else:
        tweet_data['main_tweet'] = ""
        tweet_data['main_tweet_id'] = main_tweet_id
        tweet_data['verified'] = False
        tweet_data['followers'] = 0
    tweet_data['replies'] = json.dumps(replies) 
    
    delta = delta.append(pd.DataFrame([tweet_data]))
    delta.to_csv('./project-data/%s.csv'%file, index=False)
 
    

file_open.close()



Rate limit exceeded. Sleeping for 724 seconds.


In [ ]:
# delta = pd.read_csv('./project-data/%s.csv'%file,keep_default_na=False)
# label = open('./project-data/dev.label.txt', 'r')
# label_data = label.readlines()
# delta
# delta = delta.iloc[:, 1:]


# label_data = [x.strip() for x in label_data]
delta["main_tweet"] = delta["main_tweet"].astype(str)
delta['main_tweet'].replace('nan', '', inplace=True)
# delta['label'] = label_data

delta.to_csv('./project-data/%s.csv'%file, index=False)